# Xavier Deep network version

# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.6200, train_loss: 2.07296610, train_accuracy: 0.3672, test_Accuracy: 0.2060
Epoch: [ 0] [    1/  468] time: 2.1840, train_loss: 1.98973250, train_accuracy: 0.4922, test_Accuracy: 0.4592
Epoch: [ 0] [    2/  468] time: 2.7820, train_loss: 1.79307199, train_accuracy: 0.7266, test_Accuracy: 0.6365
Epoch: [ 0] [    3/  468] time: 3.3850, train_loss: 1.55289590, train_accuracy: 0.7031, test_Accuracy: 0.6821
Epoch: [ 0] [    4/  468] time: 4.0160, train_loss: 1.24313354, train_accuracy: 0.7109, test_Accuracy: 0.6572
Epoch: [ 0] [    5/  468] time: 4.5480, train_loss: 1.05351233, train_accuracy: 0.7656, test_Accuracy: 0.7052
Epoch: [ 0] [    6/  468] time: 5.0830, train_loss: 0.79410684, train_accuracy: 0.7656, test_Accuracy: 0.6976
Epoch: [ 0] [    7/  468] time: 5.6131, train_loss: 0.71339667, train_accuracy: 0.8047, test_Accuracy: 0.7586
Epoch: [ 0] [    8/  468] time: 6.1400,

Epoch: [ 0] [   74/  468] time: 44.0860, train_loss: 0.25231022, train_accuracy: 0.9297, test_Accuracy: 0.9222
Epoch: [ 0] [   75/  468] time: 44.6290, train_loss: 0.26006931, train_accuracy: 0.9219, test_Accuracy: 0.9260
Epoch: [ 0] [   76/  468] time: 45.2141, train_loss: 0.26549917, train_accuracy: 0.9297, test_Accuracy: 0.9239
Epoch: [ 0] [   77/  468] time: 45.7860, train_loss: 0.18340820, train_accuracy: 0.9453, test_Accuracy: 0.9150
Epoch: [ 0] [   78/  468] time: 46.3090, train_loss: 0.29419795, train_accuracy: 0.8906, test_Accuracy: 0.9198
Epoch: [ 0] [   79/  468] time: 46.8690, train_loss: 0.31250411, train_accuracy: 0.9219, test_Accuracy: 0.9282
Epoch: [ 0] [   80/  468] time: 47.4220, train_loss: 0.14052300, train_accuracy: 0.9688, test_Accuracy: 0.9326
Epoch: [ 0] [   81/  468] time: 47.9790, train_loss: 0.14809769, train_accuracy: 0.9688, test_Accuracy: 0.9337
Epoch: [ 0] [   82/  468] time: 48.5150, train_loss: 0.22265276, train_accuracy: 0.9453, test_Accuracy: 0.9279
E

Epoch: [ 0] [  148/  468] time: 84.9850, train_loss: 0.16724753, train_accuracy: 0.9531, test_Accuracy: 0.9483
Epoch: [ 0] [  149/  468] time: 85.5510, train_loss: 0.15006250, train_accuracy: 0.9531, test_Accuracy: 0.9502
Epoch: [ 0] [  150/  468] time: 86.0600, train_loss: 0.21851031, train_accuracy: 0.9531, test_Accuracy: 0.9511
Epoch: [ 0] [  151/  468] time: 86.5231, train_loss: 0.21135142, train_accuracy: 0.9297, test_Accuracy: 0.9532
Epoch: [ 0] [  152/  468] time: 87.1170, train_loss: 0.16381854, train_accuracy: 0.9609, test_Accuracy: 0.9532
Epoch: [ 0] [  153/  468] time: 87.6471, train_loss: 0.19881748, train_accuracy: 0.9453, test_Accuracy: 0.9524
Epoch: [ 0] [  154/  468] time: 88.1860, train_loss: 0.14133179, train_accuracy: 0.9453, test_Accuracy: 0.9496
Epoch: [ 0] [  155/  468] time: 88.6460, train_loss: 0.15566699, train_accuracy: 0.9531, test_Accuracy: 0.9468
Epoch: [ 0] [  156/  468] time: 89.1810, train_loss: 0.09260961, train_accuracy: 0.9688, test_Accuracy: 0.9453
E

Epoch: [ 0] [  222/  468] time: 126.9470, train_loss: 0.18550387, train_accuracy: 0.9297, test_Accuracy: 0.9583
Epoch: [ 0] [  223/  468] time: 127.4790, train_loss: 0.13875702, train_accuracy: 0.9609, test_Accuracy: 0.9588
Epoch: [ 0] [  224/  468] time: 128.1800, train_loss: 0.09703319, train_accuracy: 0.9766, test_Accuracy: 0.9592
Epoch: [ 0] [  225/  468] time: 128.8780, train_loss: 0.11523120, train_accuracy: 0.9609, test_Accuracy: 0.9603
Epoch: [ 0] [  226/  468] time: 129.4720, train_loss: 0.15489781, train_accuracy: 0.9688, test_Accuracy: 0.9599
Epoch: [ 0] [  227/  468] time: 130.1101, train_loss: 0.06454811, train_accuracy: 0.9766, test_Accuracy: 0.9559
Epoch: [ 0] [  228/  468] time: 130.7050, train_loss: 0.07364275, train_accuracy: 0.9688, test_Accuracy: 0.9512
Epoch: [ 0] [  229/  468] time: 131.3451, train_loss: 0.08109593, train_accuracy: 0.9766, test_Accuracy: 0.9490
Epoch: [ 0] [  230/  468] time: 131.8820, train_loss: 0.14399084, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 171.0190, train_loss: 0.12335230, train_accuracy: 0.9844, test_Accuracy: 0.9642
Epoch: [ 0] [  297/  468] time: 171.6250, train_loss: 0.16499320, train_accuracy: 0.9531, test_Accuracy: 0.9643
Epoch: [ 0] [  298/  468] time: 172.2380, train_loss: 0.05222566, train_accuracy: 0.9922, test_Accuracy: 0.9633
Epoch: [ 0] [  299/  468] time: 172.8250, train_loss: 0.09088504, train_accuracy: 0.9766, test_Accuracy: 0.9609
Epoch: [ 0] [  300/  468] time: 173.3740, train_loss: 0.24009612, train_accuracy: 0.9141, test_Accuracy: 0.9614
Epoch: [ 0] [  301/  468] time: 173.9550, train_loss: 0.07219054, train_accuracy: 0.9766, test_Accuracy: 0.9601
Epoch: [ 0] [  302/  468] time: 174.5390, train_loss: 0.06463647, train_accuracy: 0.9844, test_Accuracy: 0.9587
Epoch: [ 0] [  303/  468] time: 175.1000, train_loss: 0.14290556, train_accuracy: 0.9531, test_Accuracy: 0.9596
Epoch: [ 0] [  304/  468] time: 175.7410, train_loss: 0.03129309, train_accuracy: 1.0000, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 220.9284, train_loss: 0.07010594, train_accuracy: 0.9766, test_Accuracy: 0.9611
Epoch: [ 0] [  371/  468] time: 221.6873, train_loss: 0.11954963, train_accuracy: 0.9531, test_Accuracy: 0.9627
Epoch: [ 0] [  372/  468] time: 222.4103, train_loss: 0.04568664, train_accuracy: 0.9922, test_Accuracy: 0.9608
Epoch: [ 0] [  373/  468] time: 223.1173, train_loss: 0.07847105, train_accuracy: 0.9844, test_Accuracy: 0.9592
Epoch: [ 0] [  374/  468] time: 223.7603, train_loss: 0.12567250, train_accuracy: 0.9766, test_Accuracy: 0.9575
Epoch: [ 0] [  375/  468] time: 224.3713, train_loss: 0.04500348, train_accuracy: 0.9844, test_Accuracy: 0.9598
Epoch: [ 0] [  376/  468] time: 225.0703, train_loss: 0.12583478, train_accuracy: 0.9531, test_Accuracy: 0.9619
Epoch: [ 0] [  377/  468] time: 225.7933, train_loss: 0.11606584, train_accuracy: 0.9609, test_Accuracy: 0.9605
Epoch: [ 0] [  378/  468] time: 226.5433, train_loss: 0.06774351, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 272.9319, train_loss: 0.05604490, train_accuracy: 0.9766, test_Accuracy: 0.9603
Epoch: [ 0] [  445/  468] time: 273.6129, train_loss: 0.04780240, train_accuracy: 0.9922, test_Accuracy: 0.9637
Epoch: [ 0] [  446/  468] time: 274.3469, train_loss: 0.11859503, train_accuracy: 0.9609, test_Accuracy: 0.9660
Epoch: [ 0] [  447/  468] time: 275.0980, train_loss: 0.06332577, train_accuracy: 0.9766, test_Accuracy: 0.9681
Epoch: [ 0] [  448/  468] time: 275.7459, train_loss: 0.06270567, train_accuracy: 0.9844, test_Accuracy: 0.9699
Epoch: [ 0] [  449/  468] time: 276.4629, train_loss: 0.07870732, train_accuracy: 0.9766, test_Accuracy: 0.9671
Epoch: [ 0] [  450/  468] time: 277.2299, train_loss: 0.05496416, train_accuracy: 0.9844, test_Accuracy: 0.9635
Epoch: [ 0] [  451/  468] time: 278.0229, train_loss: 0.21635082, train_accuracy: 0.9531, test_Accuracy: 0.9629
Epoch: [ 0] [  452/  468] time: 278.7259, train_loss: 0.07686342, train_accuracy: 0.9766, test_Accuracy:

# Test accuracy : 96.26 %